# Attempting a Differential Attack on PRESENT-80 with MILP using Convex Hull of S-Box

### SETUP

Importing libraries, setting up functions, preparing the encryption standard as guided with simple_spn.ipynb

In [1]:
import secrets  # for randbits
from time import time
import numpy as np
import matplotlib.pyplot as plt
from inspect import signature
from itertools import product
from functools import reduce
from sage.crypto.block_cipher.present import PRESENT, PRESENT_KS

In [2]:
tobits= lambda x, nbits: [*map(int, format(x, "0%db"%nbits)[::-1])]     # little endian

frombits= lambda y: sum(bit*(1<<i) for i,bit in enumerate(y))

def perm(pbox, cipbits):
    assert len(cipbits) == len(pbox)
    return [cipbits[pbox[i]] for i in range(len(cipbits))]


def get_ddt(sbox):
    l= len(sbox)
    ddt= np.zeros((l,l), dtype=int)
    for i,x in enumerate(sbox):
        for j,y in enumerate(sbox):
            ddt[i^^j][x^^y]+= 1
    return ddt


def get_sbox_hrep(sbox, threshold:int=0):
    assert threshold >= 0, "Threshold must be a positive integer"
    assert threshold <= len(sbox), "Threshold= `threshold` is larger than SBOX!"
    ddt = get_ddt(sbox)
    l = len(sbox)
    lb = int(l).bit_length()-1
    assert l == 1<<lb, "Size of SBOX isn't a power of two!"   
    # each successive bit carries an extra weight of *2 => l= max(weight) of bits

    space = []
    for x in range(l):
        for y in range(l):
            if ddt[x,y] <= threshold: continue
            xb = tobits(x, lb)
            yb = tobits(y, lb)
            space.append((*xb, *yb))
    
    p = Polyhedron(vertices=space)
    return p.Hrepresentation()


def hrep_to_ineq(hrep, lin_var):
    assert len(lin_var) == len(hrep[0]) - 1, \
        "Number of arguments does not match dimensionality of `hrep`"
    # hrep includes constant b in iterable as element 0
    return [
        sum(var*coeff for coeff,var in zip([*ieq][1:], lin_var)) + ieq.b() >= 0 
        for ieq in hrep
    ]
    # 1.    The function pushes '*' multiply operator. MIP linear function (lf) 
    #       attaches real constants to variables for storage as lf object. 
    # 2.    '>=', __geq__ has been overwritten to read 2 lf and output some 
    #       linear constraint (lc), thus NOT a boolean
    # 3.    Returns a list of constraints


def toblks(arr, blocklen:int):
    assert len(arr)%blocklen ==0, "`blocklen` isn't a divisor of `len(arr)`"
    return [arr[i : i + blocklen] for i in range(0, len(arr), blocklen)]


def get_boolexpr_hrep(boolfunc):
    """
    Get H-Representation of points representing
    (a0,a1,...,an, boolfunc(a0,a1,...,an))
    
    boolfunc is a boolean function of n-args with a boolean output
    """
    nargs = len(signature(boolfunc).parameters)
    space = []
    for nb in product([0,1], repeat=nargs): # all possible binary vectors in n-d
        space.append((*nb, boolfunc(*nb)))
        
    p = Polyhedron(vertices=space)  # entire space based on the defined bool func
    return p.Hrepresentation()
    

prod= lambda arr: reduce(lambda x,y: x*y, arr)

In [3]:
class VarGen:
    
    """
    Wrapper class over `solver.new_variable`
    to provide the `gen` method
    """
    
    def __init__(self, solver:MixedIntegerLinearProgram):
        self.vargen = solver.new_variable(integer=True)
        
    def __getitem__(self, idx):
        """Get an existing variable at index `idx`"""
        assert idx < len(self.vargen.keys())
        return self.vargen[idx]
    
    def gen(self):
        """Generates a new variable"""
        return self.vargen[len(self.vargen.keys())]


In [42]:
RDS= 4

SBOX= [0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD, 0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2]  # index= original 4-bit word
INV_SBOX= [SBOX.index(i) for i in range(16)]
DDT= get_ddt(SBOX)

PBOX= [ (16*i) %63 for i in range(63)] + [63]
INV_PBOX= [PBOX.index(i) for i in range(64)]

Image taken from *"Encyclopedia of Cryptography and Security"* by **Van Tilborg et. al**.

<p align= "center">
    <img height= 700 width= 875 src= "PRESENT-cipher.png">
</p>

### Obtaining the differential characteristic at 3 rounds

Creating objects, structures, variables

In [63]:
hrep_sbox= get_sbox_hrep(SBOX,2)    # optional threshold argument for the minimum count of differential in DDT
hrep_activeS= get_boolexpr_hrep(lambda a,b,c,d: a or b or c or d)

solver= MixedIntegerLinearProgram(maximization=False)   # objective to MINIMIZE count of active S-boxes
vargen= VarGen(solver)

pt= [vargen.gen() for i in range(64)]  # generates a new variable point with each indexing imposed
ct= pt[:]       # first round of ciphertext = plaintext
sbox_ins= []    # Saves all s-box input var
sbox_outs= []   # Saves all s-box output var

Adding the constraints

In [64]:
# only non-trivial solutions
solver.add_constraint(sum(pt) >=1)

In [65]:
# for each round, add the constraints associated to permutated bit
for rd in range(RDS-1):

    # saves the cipher bit-variable order before entering s-box
    sbox_ins.extend(toblks(ct, 4))  
    # each element is block of cipher => len(sbox_ins)= total number of sboxes

    # a fresh blk at end of each sbox
    sbox_out= [vargen.gen() for i in range(64)] 
    # save the assigned constraint variables in this sbox output
    sbox_outs.extend(toblks(sbox_out, 4))

    for c_blk, out_blk in zip(toblks(ct, 4), toblks(sbox_out, 4)):  # for each sbox, 
        for c in hrep_to_ineq(hrep_sbox, c_blk + out_blk):          # generate the ieqs
            solver.add_constraint(c)                                # and add to constraints

    # permutate before next round
    ct= perm(PBOX, sbox_out)

In [66]:
active_s= [vargen.gen() for _ in range(len(sbox_ins))]

# add constraints to enable active s-box
for sbox_in, active in zip(sbox_ins, active_s):
    for c in hrep_to_ineq(hrep_activeS, sbox_in + [active]):
        solver.add_constraint(c)

Set objective function, and obtain objective value

In [67]:
objective= sum(active_s)
solver.set_objective(objective)

# solver.show()
min_s=  solver.solve()
print("Minimum Active S-Boxes:", min_s)

Minimum Active S-Boxes: 4.0


Retrieve stored values of each round

In [68]:
d_in= [*map(int, solver.get_values(pt))]
d_out= [*map(int, solver.get_values(ct))]

sbox_i=[]
for inp in toblks(sbox_ins, 16):     # every 4 sublists is 1 chain of ciphertext
    sbox_i+= [[*map(int, solver.get_values( sum(inp, [])) )]]
sbox_o=[]
for oup in toblks(sbox_outs, 16):
    sbox_o+= [[*map(int, solver.get_values( sum(oup, [])) )]]
# sum(nested list, []) converts sum into joining of lists inside an iterable


print("Plaintext Difference: ")
print("\t", " ".join( ''.join(x) for x in toblks([*map(str, d_in)], 4) ).replace('0','.'))

print("\nS-Box I/O: ")
for i,o in zip(sbox_i, sbox_o): 
    print(" I:\t", " ".join( ''.join(x) for x in toblks([*map(str, i)], 4) ).replace('0','.'))
    print(" O:\t", " ".join( ''.join(x) for x in toblks([*map(str, o)], 4) ).replace('0','.'))
    print(" "+ "_"*39 + "PERMUTATE" + "_"*39)


print("\nCiphertext Difference: ")
print("\t", " ".join( ''.join(x) for x in toblks([*map(str, d_out)], 4) ).replace('0','.'))

Plaintext Difference: 
	 .... 1..1 .... .... .... .... .... .... .... .... .... .... .... 1111 .... ....

S-Box I/O: 
 I:	 .... 1..1 .... .... .... .... .... .... .... .... .... .... .... 1111 .... ....
 O:	 .... ..1. .... .... .... .... .... .... .... .... .... .... .... ..1. .... ....
 _______________________________________PERMUTATE_______________________________________
 I:	 .... .... .... .... .... .... 1..1 .... .... .... .... .... .... .... .... ....
 O:	 .... .... .... .... .... .... ..1. .... .... .... .... .... .... .... .... ....
 _______________________________________PERMUTATE_______________________________________
 I:	 .... .... .... .... .... .... .... .... .... .... .1.. .... .... .... .... ....
 O:	 .... .... .... .... .... .... .... .... .... .... 1.1. .... .... .... .... ....
 _______________________________________PERMUTATE_______________________________________

Ciphertext Difference: 
	 .... .... .... .... .... .... .... .... ..1. .... ..1. .... .... .... .... ...

Calculating the probability of trail

In [69]:
prod= lambda arr: reduce(lambda x,y: x*y, arr)
p_rd= []
for i,o in zip(sbox_i, sbox_o):
    p_rd+= [ prod( DDT[frombits(iblk), frombits(oblk)]/len(SBOX) for iblk, oblk in  zip(toblks(i,4), toblks(o,4)) )]

p_trail= prod(p_rd)
print("Probability of Differential Characteristic: ", (p_trail))
print(" - One in", int(round(1/p_trail)))

Probability of Differential Characteristic:  0.00390625
 - One in 256
